In [ ]:
# How to read in private google sheet 

import os
import pandas as pd
import streamlit as st
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = st.secrets["sheet_id"]

def read_in_google_sheet():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SPREADSHEET_ID, range="Form Responses 1")
        .execute()
    )
    values = result.get("values", [])

    df = pd.DataFrame(values[1:], columns=values[0], index=None)
    return df

  except HttpError as err:
    print(err)

workout = read_in_google_sheet()

In [ ]:
# How to read in public google sheet from viewer only link 

import streamlit as st
from streamlit_gsheets import GSheetsConnection

# Create a connection object.
conn = st.connection("gsheets", type=GSheetsConnection)

df = conn.read()

In [4]:
# getting the days of week 
import datetime as dt

workout['Timestamp'] = pd.to_datetime(workout['Timestamp'])
#workout['day'] = workout['Timestamp'].dt.day_name()
#workout['day'] 

In [7]:
# Filter to last 3 workout days 
workout['Timestamp'] = pd.to_datetime(workout['Timestamp'])
workout['date'] = workout['Timestamp'].dt.strftime('%b %d, %Y')

# Gets the last 3 items from our list
last_three = workout['date'].unique().tolist()[-3:]

threeeee = workout[workout['date'].isin(last_three)]  

In [ ]:
# Last 5 workouts 
new_workout = workout.tail(5)